This notebook demonstrates how negation and uncertainty patterns for the CheXpert labeler work. This is illustrated using a small set of sample data on weather forecasts. The text data was created using weather forecast transcripts which can be found [here](https://learnenglishteens.britishcouncil.org/sites/teens/files/weather_forecast_-_transcript_4.pdf).

First, the data has to be loaded from a Knodle dataset collection. The data consists of "phrases", which include mentions and unmentions of eight different weather labels, and "patterns", which are divided into pre-negation uncertainty, negation and post-negation uncertainty patterns.
While the mentions are used as main rules for the corresponding labels and represented in the T and Z matrices, the unmentions and the patterns are used to finetune the matches. The unmentions, like the mentions, are saved in plain text files consisting of simple keywords corresponding to the respective labels. The patterns are saved in three different text files comprised of [SemgrexPatterns](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/semgrex/SemgrexPattern.html) which are patterns for matching node and edge configurations of a dependency graph. The patterns are composed of nodes, which represent IndexedWords, and the relations between them, which represent edges in a SemanticGraph. For more detailed information, please have a look at the syntax [here](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/semgrex/SemgrexPattern.html).

In [74]:
# Imports
import os
import pandas as pd
from joblib import dump
from minio import Minio
from tqdm.auto import tqdm

# Client to access the dataset collection
client = Minio("knodle.cc", secure=False)

In [67]:
# Adjust pandas options, so tables do not get cut off
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_45262/1837502687.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Download Data

Specify the path to the directory where you want all files to be saved to as `CHEXPERT_DATA_DIR`. The default path is given below, simply adjust it if you wish to.

In [54]:
# DATA DIRECTORY -------------------------------------------------------------------------------------
CHEXPERT_DATA_DIR = os.path.join(os.getcwd(), "examples", "labeler", "chexpert")

The next step is downloading the data from Minio. In the next section of code, the `mention`, `unmention` & `pattern` directories are first created and then the text files are saved to them. In case you want to download other data, just change the path to the correct Minio folder and adjust the file names.

In [55]:
# RULE DIRECTORIES -----------------------------------------------------------------------------------
MENTION_DATA_DIR = os.path.join(CHEXPERT_DATA_DIR, "phrases", "mention")
os.makedirs(MENTION_DATA_DIR, exist_ok=True)
files_mention = [
    "clouds.txt", "cold.txt", "rain.txt", "snow.txt",
    "storm.txt", "sun.txt", "warm.txt", "wind.txt"
]
for file in tqdm(files_mention):
    client.fget_object(
        bucket_name="knodle",
        object_name=os.path.join("datasets/weather/phrases/mention/", file),
        file_path=os.path.join(MENTION_DATA_DIR, file),
    )

UNMENTION_DATA_DIR = os.path.join(CHEXPERT_DATA_DIR, "phrases", "unmention")
os.makedirs(UNMENTION_DATA_DIR, exist_ok=True)
files_unmention = [
    "rain.txt"
]
for file in tqdm(files_unmention):
    client.fget_object(
        bucket_name="knodle",
        object_name=os.path.join("datasets/weather/phrases/unmention/", file),
        file_path=os.path.join(UNMENTION_DATA_DIR, file),
    )


# PATTERN DIRECTORY ----------------------------------------------------------------------------------
PATTERNS_DIR = os.path.join(CHEXPERT_DATA_DIR, "patterns")
os.makedirs(PATTERNS_DIR, exist_ok=True)
files_patterns = [
    "pre_negation_uncertainty.txt", "negation.txt", "post_negation_uncertainty.txt"
]
for file in tqdm(files_patterns):
    client.fget_object(
        bucket_name="knodle",
        object_name=os.path.join("datasets/weather/patterns/", file),
        file_path=os.path.join(PATTERNS_DIR, file),
    )

2022-01-30 13:50:57,372 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/mention/clouds.txt -> https://knodle.cc/knodle/datasets/weather/phrases/mention/clouds.txt
2022-01-30 13:50:57,401 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/mention/clouds.txt -> https://knodle.cc/knodle/datasets/weather/phrases/mention/clouds.txt
2022-01-30 13:50:57,431 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/mention/cold.txt -> https://knodle.cc/knodle/datasets/weather/phrases/mention/cold.txt
2022-01-30 13:50:57,451 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/mention/cold.txt -> https://knodle.cc/knodle/datasets/weather/phrases/mention/cold.txt
2022-01-30 13:50:57,474 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/mention/rain.txt -> https://knodle.cc/knodle/datasets/weather/phrases/ment

2022-01-30 13:50:57,738 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/unmention/rain.txt -> https://knodle.cc/knodle/datasets/weather/phrases/unmention/rain.txt
2022-01-30 13:50:57,758 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/phrases/unmention/rain.txt -> https://knodle.cc/knodle/datasets/weather/phrases/unmention/rain.txt


2022-01-30 13:50:57,792 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/patterns/pre_negation_uncertainty.txt -> https://knodle.cc/knodle/datasets/weather/patterns/pre_negation_uncertainty.txt
2022-01-30 13:50:57,811 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/patterns/pre_negation_uncertainty.txt -> https://knodle.cc/knodle/datasets/weather/patterns/pre_negation_uncertainty.txt
2022-01-30 13:50:57,840 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/patterns/negation.txt -> https://knodle.cc/knodle/datasets/weather/patterns/negation.txt
2022-01-30 13:50:57,858 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/patterns/negation.txt -> https://knodle.cc/knodle/datasets/weather/patterns/negation.txt
2022-01-30 13:50:57,877 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/patterns/post_negation_uncertainty.txt -> https://

Following the same steps as above, the sample data, for which we use weather forecasts, is downloaded and stored. The sample data, in contrast to the other files, needs to be provided in a csv file.

In [56]:
# SAMPLE DIRECTORY -----------------------------------------------------------------------------------
SAMPLE_DIR = os.path.join(CHEXPERT_DATA_DIR, "reports")
os.makedirs(SAMPLE_DIR, exist_ok=True)
files_sample = [
    "weather_forecast.csv"
]
for file in tqdm(files_sample):
    client.fget_object(
        bucket_name="knodle",
        object_name=os.path.join("datasets/weather/reports/", file),
        file_path=os.path.join(SAMPLE_DIR, file),
    )

2022-01-30 13:51:01,343 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/reports/weather_forecast.csv -> https://knodle.cc/knodle/datasets/weather/reports/weather_forecast.csv
2022-01-30 13:51:01,369 urllib3.poolmanager INFO     Redirecting http://knodle.cc/knodle/datasets/weather/reports/weather_forecast.csv -> https://knodle.cc/knodle/datasets/weather/reports/weather_forecast.csv


Finally, the directory where you want the output matrices X, T and Z to be stored, needs to be specified.

In [57]:
# OUTPUT DIRECTORY -----------------------------------------------------------------------------------
OUTPUT_DIR = os.path.join(CHEXPERT_DATA_DIR, "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Preview Dataset & Patterns

The sample data, or X matrix as it is called within Knodle, is shown below. It consists of six lines, or documents as they are called within the labeler code, each including information about the weather.

In [58]:
pd.read_csv(os.path.join(SAMPLE_DIR, "weather_forecast.csv"), header=None)

,0
0,It is very windy and cold. There is going to be rain or snow.
1,It is rainy all day. There may be a thunderstorm in the afternoon.
2,"It is cloudy, but the weather will stay dry. So no rain today."
3,"It is cold, but snow is still not likely."
4,"The weather is acting up today, even a storm is possible."
5,The weather is getting better. No development of rain today.
6,"The clouds have cleared, it is going to be a sunny day."


There are eight different weather labels, so eight different text files with match phrases, that could be assigned to each of these sentences. The labels are: "clouds", "cold", "rain", "snow", "storm", "sun", "warm" and "wind". Take for example the first line of the data: "It is very windy and cold. There is going to be rain or snow." If we had to assign positive, negative or uncertain for each of these labels here, we would probably assign the following:
- **positive**: cold, wind
- **negative**: clouds, storm, sun, warm
- **uncertain**: rain, snow

The author of the sentence mentions the weather conditions "wind", "cold", "rain" and "snow". So the labeler will find these labels based on the match phrases in the corresponding text files. However, a mention does not always mean that the label should be positive. In the case of the example sentence, it says "there is going to be rain or snow", so there is in fact uncertainty concerning the labels "rain" and "snow". This uncertainty we want the labeler to recognize and we manage that through providing the necessary uncertainty pattern.

In [59]:
pd.read_csv(os.path.join(PATTERNS_DIR, "post_negation_uncertainty.txt"), header=None)

,0
0,# XXX cannot be excluded
1,{} < {} ({lemma:/excluded/} > {dependency:/neg/} {})
2,# May/might/would/could be XXX
3,{} > {} {lemma:/may|might|would|could/}
4,# XXX is possible
5,{} < {} {lemma:/possible/}
6,# XXX or YYY
7,{} > {dependency:/conj:or/} {}
8,{} < {dependency:/conj:or/} {}
9,# XXX is not likely


Above, the provided post-negation uncertainty patterns are shown. The last two lines of the file show the patterns responsible for "rain" and "snow" being labelled as uncertain. The first of the two patterns makes sure that "rain", the word before the "or",  is labelled as uncertain and the other one does the same for "snow", the word after the "or". As mentioned before, a detailed explanation of Semgrex can be found on the corresponding [website](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/semgrex/SemgrexPattern.html). In this example, however, the pattern is quite simple, the following explanation concerning the node relations is taken from the Semgrex website:
- A >reln B 	A is the governor of a relation reln with B
- A <reln B 	A is the dependent of a relation reln with B

In the relation "rain or snow", "rain" is the governor and "snow" the dependent, therefore we need the ">reln" relation to label "rain" as uncertain and "<reln" to label "snow" as uncertain.

Let us also have a closer look at the rest of the documents. The second one reads: "It is rainy all day. There may be a thunderstorm in the afternoon."
The human label assignment would be:
- **positive**: rain
- **negative**: clouds, cold, snow, sun, warm, wind
- **uncertain**: storm

Using the provided mentions, the labeler will find the labels "rain" and "storm". However, we want the labeler to correctly classify "storm" as uncertain, since it says "there **may** be a thunderstorm". This is accomplished with the following pattern: "{ } > { } {lemma:/may|might|would|could/}". In our example sentence "thunderstorm" is the governor of the relation, therefore we need ">". The "be" is represented by "{ }" and followed by the node holding our lemma for "may".

-------------------------------------------
Let us continue with the third document: "It is cloudy, but the weather will stay dry. So no rain today."
The human label assignment would be:
- **positive**: clouds,
- **negative**: cold, rain, snow, storm, sun, warm, wind
- **uncertain**: /

The labels the labeler will find using the mentions are "clouds" and "rain". However, contrary to the first two examples, the labeler won't actually need the negation or uncertainty patterns, since "dry" is an unmention provided for "rain".

-------------------------------------------
The fourth document reads: "It is cold, but snow is still not likely."
The human label assignment would be:
- **positive**: cold
- **negative**: clouds, rain, storm, sun, warm, wind
- **uncertain**: snow

The labeler will find "cold" and "snow" using the mentions. The uncertainty pattern responsible for the correct labelling of "snow" as uncertain is "{ } < { } ({lemma:/likely/} > {dependency:/neg/} { })". "snow" is the dependent, therefore "<" is needed as dependency and inside the parentheses the "not likely" is depicted.

-------------------------------------------
The fifth document states: "The weather is acting up today, even a storm is possible."
The human label assignment would be:
- **positive**: /
- **negative**: clouds, cold, rain, snow, sun, warm, wind
- **uncertain**: storm

The following pattern is the reason for the labeler to correctly label "storm" as uncertain: "{ } < { } {lemma:/possible/}". Since "storm" is the dependent, we again need "<". The "is" is represented by "{ }" and followed by the node holding our lemma for "possible".

-------------------------------------------
In the sixth document it reads: "The weather is getting better. No development of rain today."
The human label assignment would be:
- **positive**: /
- **negative**: clouds, cold, rain, snow, storm, sun, warm, wind
- **uncertain**: /

The only mention found by the labeler corresponds to "rain". However, we need the labeler to classify rain as negative, since it says "No development of rain". This is accomplished using a negation pattern, please have a look at all the negation patterns below. The one used in this case is "{ } < {dependency:/nmod:of/} ({lemma:/development/} > { } {lemma:/no/})". "rain" is a dependent of "development" and the dependency can be specified with "nmod:of". Inside the parentheses the dependency between "development" (the governor) and "no" is shown.

-------------------------------------------
The final document states: "The clouds have cleared, it is going to be a sunny day."
The human label assignment would be:
- **positive**: sun
- **negative**: clouds, cold, rain, snow, storm, warm, wind
- **uncertain**: /

The labeler will find the mentions for "clouds" and "sun", but we only want the latter to be labeled positive. The negation of "clouds" can be achieved with the help of the following negation pattern: "{ } < { } {lemma:/clear/}". Since "clouds" is the dependent we need "<" to express the correct dependency and "{ } {lemma:/clear/}" matches "have cleared".

In [60]:
pd.read_csv(os.path.join(PATTERNS_DIR, "negation.txt"), header=None)

,0
0,# No development of XXX
1,{} < {dependency:/nmod:of/} ({lemma:/development/} > {} {lemma:/no/})
2,# XXX has/have cleared
3,{} < {} {lemma:/clear/}


Have a look at the CheXpert sample reports as well and their corresponding negation or uncertainty patterns. A very helpful website for figuring out which term is the governor and which the dependent or also for getting part-of-speech tags for a text is the [CoreNLP website](https://corenlp.run/), referring to it when creating patterns yourself can be quite useful.

## Labelling

Now, that the data is loaded, use the adjusted [CheXpert labeler](https://github.com/stanfordmlgroup/chexpert-labeler) in Knodle to label the weather forecasts. The labeler is started by initiating the `Labeler()` class, followed by running the associated `label()` function. Since we are not using the original CheXpert data, `chexpert_bool` is set to `False`. And the `config_pattern_tutorial.py` file is specified as config file for the labeler.
You can choose yourself how you want the uncertain labels to be used. The default of the labeler is to transform all uncertain labels to positive labels, but this can be changed through the `uncertain` argument in the `label()` function of the `labeler`. Simply specify "-1" if you want to keep the uncertain labels or "0" if you want the uncertain labels to be changed to negative labels. Unfortunately the other modules e.g. trainers in Knodle can only handle Z matrices containing zeros and ones as of now.

For running the code, please check the identically named python file and run it as explained below.

Unfortunately the code cannot be run from this notebook, but must be run in a python file from the terminal, because of NegBio. Please run these lines of code in your terminal and adjust the paths accordingly beforehand:

**0) Go to home directory**
`cd`

**1) Clone the repository**
`git clone https://github.com/ncbi-nlp/NegBio.git`

**2) Add the NegBio directory to your PYTHONPATH**
`export PYTHONPATH="${PYTHONPATH}:<insert path to NegBio directory>"`

**3) Create the virtual environment**
`cd ~<insert path to knodle/knodle/labeler/CheXpert directory>
conda env create -f environment.yml`

**4) Activate the virtual environment**
`conda activate chexpert-label`

**5) Install NLTK data**
`python -m nltk.downloader universal_tagset punkt wordnet`

**6) Go to knodle directory**
`cd ~<insert path to knodle>`

**7) Run the file**
`python examples/labeler/chexpert/chexpert_patterns_tutorial.py`

## Results

The resulting matrices X, T and Z are printed below.

In [71]:
X_matrix = os.path.join(OUTPUT_DIR, "X_matrix.csv")
pd.read_csv(X_matrix, header=None)

,0
0,It is very windy and cold. There is going to be rain or snow.
1,It is rainy all day. There may be a thunderstorm in the afternoon.
2,"It is cloudy, but the weather will stay dry. So no rain today."
3,"It is cold, but snow is still not likely."
4,"The weather is acting up today, even a storm is possible."
5,The weather is getting better. No development of rain today.
6,"The clouds have cleared, it is going to be a sunny day."


In [72]:
T_matrix = os.path.join(OUTPUT_DIR, "T_matrix.csv")
pd.read_csv(T_matrix, header=None, dtype=int)

,0,1,2,3,4,5,6,7
0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0
7,0,0,1,0,0,0,0,0
8,0,0,1,0,0,0,0,0
9,0,0,0,1,0,0,0,0


Note, that in this case the argument "uncertain" was set to "-1", therefore uncertainty is displayed in the matrix below.

In [88]:
Z_matrix = os.path.join(OUTPUT_DIR, "Z_matrix.csv")
pd.read_csv(Z_matrix, header=None, dtype=int)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0


## Save files in lib format

In [84]:
dump(X_matrix, os.path.join(OUTPUT_DIR, "weather_samples_x.lib"))
dump(T_matrix, os.path.join(OUTPUT_DIR, "weather_mapping_rules_labels_t.lib"))
dump(Z_matrix, os.path.join(OUTPUT_DIR, "weather_rule_matches_z.lib"))

['/home/elisabear/PycharmProjects/knodle/examples/labeler/chexpert/output/weather_rule_matches_z.lib']